# Gradient Boosting Demo: All Three Engines

This notebook demonstrates the `boost_tree()` model specification with all three supported engines:
- **XGBoost**: Fast, scalable gradient boosting
- **LightGBM**: Microsoft's efficient gradient boosting framework
- **CatBoost**: Yandex's gradient boosting with excellent categorical support

## Key Features:
- Unified API across all engines
- 8 tunable parameters
- Feature importance extraction
- Early stopping support
- Comprehensive performance metrics
- Side-by-side engine comparison

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from py_parsnip import boost_tree
from py_workflows import workflow
from py_rsample import initial_split, vfold_cv
from py_tune import tune, tune_grid, grid_regular, finalize_workflow
from py_yardstick import metric_set, rmse, mae, r_squared
import time

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Introduction to Gradient Boosting

Gradient boosting builds an ensemble of weak learners (typically decision trees) sequentially, where each new tree tries to correct the errors of the previous ones.

### The 8 Tunable Parameters in boost_tree():

1. **trees**: Number of boosting iterations (n_estimators)
2. **tree_depth**: Maximum tree depth
3. **learn_rate**: Step size shrinkage (0 to 1)
4. **mtry**: Number of features to sample per split
5. **min_n**: Minimum samples in leaf node
6. **loss_reduction**: Minimum loss reduction for split
7. **sample_size**: Fraction of observations per tree (0 to 1)
8. **stop_iter**: Early stopping rounds

### Engine-Specific Parameter Mappings:

| Parameter | XGBoost | LightGBM | CatBoost |
|-----------|---------|----------|----------|
| trees | n_estimators | n_estimators | iterations |
| tree_depth | max_depth | max_depth | depth |
| learn_rate | learning_rate | learning_rate | learning_rate |
| mtry | colsample_bytree | colsample_bytree | rsm |
| min_n | min_child_weight | min_data_in_leaf | min_data_in_leaf |
| loss_reduction | gamma | min_split_gain | (not supported) |
| sample_size | subsample | subsample | subsample |
| stop_iter | early_stopping_rounds | early_stopping_rounds | early_stopping_rounds |

## 2. Generate Sample Data

Create a synthetic regression dataset with non-linear relationships that benefit from gradient boosting.

In [ ]:
# Generate synthetic data with non-linear relationships
n = 500
data = pd.DataFrame({
    'x1': np.random.uniform(0, 10, n),
    'x2': np.random.uniform(0, 10, n),
    'x3': np.random.uniform(0, 10, n),
    'x4': np.random.uniform(0, 10, n),
    'x5': np.random.uniform(0, 10, n),
    'x6': np.random.randint(0, 5, n),  # Categorical-like
})

# Create target with complex non-linear relationships
data['y'] = (
    2 * data['x1'] +
    3 * np.sin(data['x2']) +
    0.5 * data['x3'] ** 2 +
    1.5 * np.log(data['x4'] + 1) +
    0.8 * data['x5'] * data['x6'] +
    np.random.normal(0, 2, n)
)

print("Data shape:", data.shape)
print("\nFirst few rows:")
print(data.head())
print("\nTarget statistics:")
print(data['y'].describe())

In [ ]:
# Split into train and test
train = data.iloc[:400].copy()
test = data.iloc[400:].copy()

print(f"Train: {train.shape}, Test: {test.shape}")

## 3. XGBoost Engine

XGBoost (Extreme Gradient Boosting) is one of the most popular gradient boosting implementations, known for its speed and performance.

### 3.1 Basic XGBoost Model with All 8 Parameters

In [ ]:
# Create XGBoost specification with all parameters
spec_xgb = boost_tree(
    trees=200,              # Number of boosting rounds
    tree_depth=6,           # Maximum tree depth
    learn_rate=0.1,         # Learning rate (eta)
    mtry=4,                 # Features to sample per split
    min_n=10,               # Minimum child weight
    loss_reduction=0.01,    # Gamma (minimum loss reduction)
    sample_size=0.8,        # Row sampling fraction
    stop_iter=20,           # Early stopping rounds
    engine="xgboost"
).set_mode("regression")

print("XGBoost Model Specification:")
print(spec_xgb)

In [ ]:
# Fit the model
start_time = time.time()
fit_xgb = spec_xgb.fit(train, "y ~ x1 + x2 + x3 + x4 + x5 + x6")
xgb_train_time = time.time() - start_time

print(f"Model fitted in {xgb_train_time:.2f} seconds")

### 3.2 XGBoost Predictions and Evaluation

In [ ]:
# Predict on test data
predictions_xgb = fit_xgb.predict(test, type="numeric")
print("Predictions (first 10):")
print(predictions_xgb.head(10))

In [ ]:
# Evaluate on test data
fit_xgb = fit_xgb.evaluate(test)
print("Model evaluated on test data!")

### 3.3 XGBoost Feature Importance

In [ ]:
# Extract comprehensive outputs
outputs_xgb, feature_importance_xgb, stats_xgb = fit_xgb.extract_outputs()

print("="*60)
print("XGBoost Feature Importance")
print("="*60)
print(feature_importance_xgb)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_xgb['variable'], feature_importance_xgb['importance'])
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('XGBoost Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.4 XGBoost Performance Metrics

In [ ]:
# Display performance metrics by split
print("="*60)
print("XGBoost Performance Metrics")
print("="*60)

metric_cols = ['rmse', 'mae', 'r_squared', 'mape']
metrics_subset = stats_xgb[stats_xgb['metric'].isin(metric_cols)]

if not metrics_subset.empty:
    metrics_pivot = metrics_subset.pivot(index='metric', columns='split', values='value')
    print(metrics_pivot)
    print("\nInterpretation:")
    print("- Lower RMSE/MAE is better")
    print("- Higher R-squared is better (closer to 1)")
    print("- Compare train vs test to assess overfitting")

In [ ]:
# Plot predictions vs actuals
test_outputs = outputs_xgb[outputs_xgb['split'] == 'test'].copy()

plt.figure(figsize=(10, 6))
plt.scatter(test_outputs['actuals'], test_outputs['forecast'], alpha=0.6, s=50)
plt.plot([test_outputs['actuals'].min(), test_outputs['actuals'].max()],
         [test_outputs['actuals'].min(), test_outputs['actuals'].max()],
         'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.title('XGBoost: Predictions vs Actuals (Test Set)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. LightGBM Engine

LightGBM is Microsoft's gradient boosting framework that uses histogram-based algorithms for faster training and lower memory usage.

### 4.1 Basic LightGBM Model with All 8 Parameters

In [ ]:
# Create LightGBM specification with same parameters for comparison
spec_lgb = boost_tree(
    trees=200,              # Number of boosting rounds
    tree_depth=6,           # Maximum tree depth
    learn_rate=0.1,         # Learning rate
    mtry=4,                 # Features to sample per split
    min_n=10,               # Minimum data in leaf
    loss_reduction=0.01,    # Minimum split gain
    sample_size=0.8,        # Row sampling fraction
    stop_iter=20,           # Early stopping rounds
    engine="lightgbm"
).set_mode("regression")

print("LightGBM Model Specification:")
print(spec_lgb)

In [ ]:
# Fit the model
start_time = time.time()
fit_lgb = spec_lgb.fit(train, "y ~ x1 + x2 + x3 + x4 + x5 + x6")
lgb_train_time = time.time() - start_time

print(f"Model fitted in {lgb_train_time:.2f} seconds")

### 4.2 LightGBM Predictions and Evaluation

In [ ]:
# Predict and evaluate
predictions_lgb = fit_lgb.predict(test, type="numeric")
fit_lgb = fit_lgb.evaluate(test)

print("LightGBM predictions (first 10):")
print(predictions_lgb.head(10))

### 4.3 LightGBM Feature Importance

In [ ]:
# Extract comprehensive outputs
outputs_lgb, feature_importance_lgb, stats_lgb = fit_lgb.extract_outputs()

print("="*60)
print("LightGBM Feature Importance")
print("="*60)
print(feature_importance_lgb)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_lgb['variable'], feature_importance_lgb['importance'])
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('LightGBM Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 4.4 LightGBM Performance Metrics

In [ ]:
# Display performance metrics by split
print("="*60)
print("LightGBM Performance Metrics")
print("="*60)

metrics_subset_lgb = stats_lgb[stats_lgb['metric'].isin(metric_cols)]

if not metrics_subset_lgb.empty:
    metrics_pivot_lgb = metrics_subset_lgb.pivot(index='metric', columns='split', values='value')
    print(metrics_pivot_lgb)

In [ ]:
# Plot predictions vs actuals
test_outputs_lgb = outputs_lgb[outputs_lgb['split'] == 'test'].copy()

plt.figure(figsize=(10, 6))
plt.scatter(test_outputs_lgb['actuals'], test_outputs_lgb['forecast'], alpha=0.6, s=50, color='green')
plt.plot([test_outputs_lgb['actuals'].min(), test_outputs_lgb['actuals'].max()],
         [test_outputs_lgb['actuals'].min(), test_outputs_lgb['actuals'].max()],
         'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.title('LightGBM: Predictions vs Actuals (Test Set)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. CatBoost Engine

CatBoost is Yandex's gradient boosting library with built-in support for categorical features and robust default parameters.

### 5.1 Basic CatBoost Model with All 8 Parameters

In [ ]:
# Create CatBoost specification with same parameters for comparison
spec_cat = boost_tree(
    trees=200,              # Number of boosting iterations
    tree_depth=6,           # Maximum tree depth
    learn_rate=0.1,         # Learning rate
    mtry=4,                 # Features to sample per split (rsm)
    min_n=10,               # Minimum data in leaf
    loss_reduction=0.01,    # Not supported in CatBoost
    sample_size=0.8,        # Row sampling fraction
    stop_iter=20,           # Early stopping rounds
    engine="catboost"
).set_mode("regression")

print("CatBoost Model Specification:")
print(spec_cat)
print("\nNote: loss_reduction parameter is not supported by CatBoost and will be ignored")

In [ ]:
# Fit the model
start_time = time.time()
fit_cat = spec_cat.fit(train, "y ~ x1 + x2 + x3 + x4 + x5 + x6")
cat_train_time = time.time() - start_time

print(f"Model fitted in {cat_train_time:.2f} seconds")

### 5.2 CatBoost Predictions and Evaluation

In [ ]:
# Predict and evaluate
predictions_cat = fit_cat.predict(test, type="numeric")
fit_cat = fit_cat.evaluate(test)

print("CatBoost predictions (first 10):")
print(predictions_cat.head(10))

### 5.3 CatBoost Feature Importance

In [ ]:
# Extract comprehensive outputs
outputs_cat, feature_importance_cat, stats_cat = fit_cat.extract_outputs()

print("="*60)
print("CatBoost Feature Importance")
print("="*60)
print(feature_importance_cat)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_cat['variable'], feature_importance_cat['importance'])
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('CatBoost Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 5.4 CatBoost Performance Metrics

In [ ]:
# Display performance metrics by split
print("="*60)
print("CatBoost Performance Metrics")
print("="*60)

metrics_subset_cat = stats_cat[stats_cat['metric'].isin(metric_cols)]

if not metrics_subset_cat.empty:
    metrics_pivot_cat = metrics_subset_cat.pivot(index='metric', columns='split', values='value')
    print(metrics_pivot_cat)

In [ ]:
# Plot predictions vs actuals
test_outputs_cat = outputs_cat[outputs_cat['split'] == 'test'].copy()

plt.figure(figsize=(10, 6))
plt.scatter(test_outputs_cat['actuals'], test_outputs_cat['forecast'], alpha=0.6, s=50, color='purple')
plt.plot([test_outputs_cat['actuals'].min(), test_outputs_cat['actuals'].max()],
         [test_outputs_cat['actuals'].min(), test_outputs_cat['actuals'].max()],
         'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.title('CatBoost: Predictions vs Actuals (Test Set)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Side-by-Side Engine Comparison

Compare all three engines on the same dataset to understand their relative strengths.

### 6.1 Performance Metrics Comparison Table

In [ ]:
# Create comprehensive comparison table
comparison_data = []

# Extract test metrics for each engine
for engine_name, stats_df in [('XGBoost', stats_xgb), ('LightGBM', stats_lgb), ('CatBoost', stats_cat)]:
    test_metrics = stats_df[(stats_df['split'] == 'test') & (stats_df['metric'].isin(metric_cols))]
    
    metrics_dict = {'Engine': engine_name}
    for _, row in test_metrics.iterrows():
        metrics_dict[row['metric'].upper()] = row['value']
    
    comparison_data.append(metrics_dict)

comparison_df = pd.DataFrame(comparison_data)

print("="*80)
print("PERFORMANCE COMPARISON: TEST SET METRICS")
print("="*80)
print(comparison_df.to_string(index=False))
print("\nLower RMSE/MAE/MAPE and higher R_SQUARED are better")

In [ ]:
# Add training time comparison
time_comparison = pd.DataFrame({
    'Engine': ['XGBoost', 'LightGBM', 'CatBoost'],
    'Training Time (seconds)': [xgb_train_time, lgb_train_time, cat_train_time]
})

print("\n" + "="*80)
print("TRAINING TIME COMPARISON")
print("="*80)
print(time_comparison.to_string(index=False))

### 6.2 Visual Performance Comparison

In [ ]:
# Plot RMSE and R-squared comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# RMSE comparison
rmse_values = comparison_df['RMSE'].values
engines = comparison_df['Engine'].values
colors = ['#1f77b4', '#2ca02c', '#9467bd']

axes[0].bar(engines, rmse_values, color=colors, alpha=0.7)
axes[0].set_ylabel('RMSE (lower is better)', fontsize=12)
axes[0].set_title('RMSE Comparison', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# R-squared comparison
r2_values = comparison_df['R_SQUARED'].values

axes[1].bar(engines, r2_values, color=colors, alpha=0.7)
axes[1].set_ylabel('R-squared (higher is better)', fontsize=12)
axes[1].set_title('R-squared Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylim([r2_values.min() * 0.95, 1.0])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 6.3 Feature Importance Comparison

In [ ]:
# Compare feature importances across engines
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Normalize importances to 0-1 scale for fair comparison
for idx, (importance_df, engine_name, color) in enumerate([
    (feature_importance_xgb, 'XGBoost', '#1f77b4'),
    (feature_importance_lgb, 'LightGBM', '#2ca02c'),
    (feature_importance_cat, 'CatBoost', '#9467bd')
]):
    # Normalize to sum to 1
    normalized_importance = importance_df.copy()
    total = normalized_importance['importance'].sum()
    if total > 0:
        normalized_importance['importance'] = normalized_importance['importance'] / total
    
    axes[idx].barh(normalized_importance['variable'], normalized_importance['importance'], color=color, alpha=0.7)
    axes[idx].set_xlabel('Normalized Importance', fontsize=11)
    axes[idx].set_title(f'{engine_name}', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.suptitle('Feature Importance Comparison Across Engines', fontsize=15, fontweight='bold', y=1.02)
plt.show()

print("Note: Importances are normalized to sum to 1 for fair comparison")

### 6.4 When to Use Each Engine

#### XGBoost:
- **Strengths**: 
  - Mature, well-documented, widely used
  - Excellent performance on structured data
  - Rich ecosystem and community support
  - Strong regularization options (gamma, lambda, alpha)
- **Best for**: General-purpose gradient boosting, competitions, production systems
- **Considerations**: Can be memory-intensive for large datasets

#### LightGBM:
- **Strengths**:
  - Faster training speed (histogram-based)
  - Lower memory consumption
  - Good performance on large datasets
  - Handles sparse data efficiently
- **Best for**: Large datasets (>10k rows), speed-critical applications
- **Considerations**: May overfit on small datasets (<1k rows)

#### CatBoost:
- **Strengths**:
  - Excellent handling of categorical features (no preprocessing needed)
  - Robust default parameters (less tuning required)
  - Good out-of-the-box performance
  - Less prone to overfitting
- **Best for**: Datasets with many categorical features, quick prototyping
- **Considerations**: Can be slower than LightGBM on very large datasets

## 7. Hyperparameter Tuning with tune_grid

Demonstrate automated hyperparameter tuning for gradient boosting models.

In [ ]:
# Create a model specification with tunable parameters
spec_tune = boost_tree(
    trees=tune('trees'),
    tree_depth=tune('tree_depth'),
    learn_rate=tune('learn_rate'),
    engine="xgboost"
).set_mode("regression")

print("Model specification with tunable parameters:")
print(spec_tune)

In [ ]:
# Create workflow
wf_tune = workflow().add_formula("y ~ x1 + x2 + x3 + x4 + x5 + x6").add_model(spec_tune)

# Create cross-validation folds
folds = vfold_cv(train, v=3)

print(f"Created workflow with {len(folds)} cross-validation folds")

In [ ]:
# Define parameter grid
param_info = {
    'trees': {'range': (50, 200), 'trans': 'log'},
    'tree_depth': {'range': (3, 8)},
    'learn_rate': {'range': (0.01, 0.3), 'trans': 'log'}
}

param_grid = grid_regular(param_info, levels=3)

print(f"Parameter grid with {len(param_grid)} combinations:")
print(param_grid)

In [ ]:
# Define metrics
metrics = metric_set(rmse, mae, r_squared)

# Run grid search
print("Running grid search... (this may take a minute)")
tune_results = tune_grid(
    wf_tune,
    folds,
    grid=param_grid,
    metrics=metrics,
    control={'save_pred': False}
)

print("\nGrid search complete!")

In [ ]:
# Show best configurations
best_configs = tune_results.show_best('rmse', n=5, maximize=False)

print("="*80)
print("TOP 5 CONFIGURATIONS BY RMSE")
print("="*80)
print(best_configs)

In [ ]:
# Select best parameters
best_params = tune_results.select_best('rmse', maximize=False)

print("Best parameters (lowest RMSE):")
print(best_params)

In [ ]:
# Visualize tuning results
all_metrics = tune_results.collect_metrics()

# Calculate mean RMSE for each config
rmse_summary = all_metrics[all_metrics['metric'] == 'rmse'].groupby('.config').agg({
    'value': ['mean', 'std']
}).reset_index()
rmse_summary.columns = ['.config', 'mean_rmse', 'std_rmse']

# Merge with parameter values
rmse_with_params = rmse_summary.merge(param_grid, on='.config')

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# RMSE vs trees (colored by learn_rate)
scatter1 = axes[0].scatter(rmse_with_params['trees'], rmse_with_params['mean_rmse'], 
                           c=rmse_with_params['learn_rate'], s=100, alpha=0.7, cmap='viridis')
axes[0].set_xlabel('Number of Trees', fontsize=12)
axes[0].set_ylabel('Mean RMSE', fontsize=12)
axes[0].set_title('RMSE vs Trees (colored by learning rate)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Learning Rate')

# RMSE vs tree_depth (colored by learn_rate)
scatter2 = axes[1].scatter(rmse_with_params['tree_depth'], rmse_with_params['mean_rmse'],
                           c=rmse_with_params['learn_rate'], s=100, alpha=0.7, cmap='viridis')
axes[1].set_xlabel('Tree Depth', fontsize=12)
axes[1].set_ylabel('Mean RMSE', fontsize=12)
axes[1].set_title('RMSE vs Tree Depth (colored by learning rate)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Learning Rate')

plt.tight_layout()
plt.show()

In [ ]:
# Finalize workflow with best parameters
final_wf = finalize_workflow(wf_tune, best_params)

# Fit on full training data
final_fit = final_wf.fit(train)

# Predict on test data
final_predictions = final_fit.predict(test)

# Calculate test metrics
test_rmse = rmse(test['y'], final_predictions['.pred'])
test_r2 = r_squared(test['y'], final_predictions['.pred'])

print("Final tuned model performance on test set:")
print(f"RMSE: {test_rmse['rmse'].values[0]:.4f}")
print(f"R-squared: {test_r2['r_squared'].values[0]:.4f}")

## 8. Extract Outputs for All Three Engines

Comprehensive overview of the three-DataFrame output structure.

### 8.1 Outputs DataFrame (Observation-level)

In [ ]:
print("="*80)
print("OUTPUTS DATAFRAME: Observation-level predictions and residuals")
print("="*80)
print("\nXGBoost Outputs (first 10 rows):")
print(outputs_xgb.head(10))
print(f"\nShape: {outputs_xgb.shape}")
print(f"Columns: {list(outputs_xgb.columns)}")

print("\nKey columns:")
print("- actuals: True target values")
print("- fitted: Model predictions on training data")
print("- forecast: Best available prediction (actuals if present, else fitted)")
print("- residuals: actuals - fitted")
print("- split: 'train' or 'test'")

### 8.2 Feature Importance DataFrame

In [ ]:
print("="*80)
print("FEATURE IMPORTANCE DATAFRAME: Variable importance scores")
print("="*80)

# Compare feature importances side-by-side
importance_comparison = pd.DataFrame({
    'Variable': feature_importance_xgb['variable'],
    'XGBoost': feature_importance_xgb['importance'].values,
    'LightGBM': feature_importance_lgb['importance'].values,
    'CatBoost': feature_importance_cat['importance'].values
})

print(importance_comparison)

print("\nInterpretation:")
print("- Higher importance = feature contributes more to predictions")
print("- Importance measures vary by engine (gain, split count, etc.)")
print("- Use for feature selection and understanding model behavior")

### 8.3 Stats DataFrame (Model-level Metrics)

In [ ]:
print("="*80)
print("STATS DATAFRAME: Model-level metrics and metadata")
print("="*80)
print("\nXGBoost Stats (performance metrics):")
print(stats_xgb[stats_xgb['metric'].isin(['rmse', 'mae', 'r_squared', 'mape', 'model_type', 'n_obs_train'])])

print("\nKey metrics:")
print("- rmse: Root Mean Squared Error")
print("- mae: Mean Absolute Error")
print("- r_squared: Coefficient of determination")
print("- mape: Mean Absolute Percentage Error")
print("\nMetrics are split by 'train' and 'test' to assess overfitting")

### 8.4 Residual Analysis

In [ ]:
# Compare residuals across engines
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (outputs_df, engine_name, color) in enumerate([
    (outputs_xgb, 'XGBoost', '#1f77b4'),
    (outputs_lgb, 'LightGBM', '#2ca02c'),
    (outputs_cat, 'CatBoost', '#9467bd')
]):
    test_data = outputs_df[outputs_df['split'] == 'test'].copy()
    
    axes[idx].scatter(test_data['forecast'], test_data['residuals'], alpha=0.6, s=50, color=color)
    axes[idx].axhline(y=0, color='red', linestyle='--', linewidth=2)
    axes[idx].set_xlabel('Predicted Values', fontsize=11)
    axes[idx].set_ylabel('Residuals', fontsize=11)
    axes[idx].set_title(f'{engine_name} Residuals', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Residual Plots: Test Set Comparison', fontsize=15, fontweight='bold', y=1.02)
plt.show()

print("Good residual plots should:")
print("- Show random scatter around zero")
print("- Have constant variance (homoscedasticity)")
print("- Show no obvious patterns or trends")

## Summary

This demo covered:

### 1. **All 8 Parameters** of boost_tree():
   - trees, tree_depth, learn_rate, mtry, min_n, loss_reduction, sample_size, stop_iter
   - Consistent API across all three engines

### 2. **Three Gradient Boosting Engines**:
   - **XGBoost**: Industry standard, excellent performance
   - **LightGBM**: Fast training, efficient memory usage
   - **CatBoost**: Great categorical handling, robust defaults

### 3. **Feature Importance**:
   - Extract and compare feature importances
   - Understand which features drive predictions
   - Different engines may rank features differently

### 4. **Early Stopping**:
   - Prevent overfitting with stop_iter parameter
   - Automatically uses validation set (last 20% of training)
   - Supported by all three engines

### 5. **Performance Comparison**:
   - Side-by-side metrics table
   - Training time comparison
   - Visual performance analysis

### 6. **Hyperparameter Tuning**:
   - Use tune_grid() for automated search
   - Cross-validation for robust evaluation
   - Visualize tuning results

### 7. **Comprehensive Outputs**:
   - **Outputs**: Observation-level predictions and residuals
   - **Feature Importance**: Variable importance scores
   - **Stats**: Model-level metrics by split

## Key Takeaways:

1. All three engines provide similar performance with proper tuning
2. LightGBM is typically fastest for large datasets
3. CatBoost requires less tuning and handles categoricals well
4. XGBoost has the most mature ecosystem and documentation
5. Use cross-validation and hyperparameter tuning for best results
6. Monitor train vs test metrics to detect overfitting
7. Feature importance helps interpret and debug models